In [ ]:
# %% [markdown]
# # Bybit DCA Bot - Execution Notebook (v9 - Modular)

# %% [markdown]
# **IMPORTANT:**
# 1. Ensure all `.py` files (`config.py`, `state_manager.py`, `utils.py`, `bybit_client.py`, `indicators.py`, `order_manager.py`, `position_manager.py`, `main_logic.py`) are in the same directory as this notebook.
# 2. Make sure required libraries are installed (`pip install pybit pandas pandas_ta python-dotenv`).
# 3. Set your Bybit Testnet API keys as environment variables OR create a `.env` file in this directory and ensure `config.py` is set up to load it.

# %%
# --- Imports ---
import time
import config         # To access parameters if needed directly (usually not needed here)
import state_manager
import bybit_client
import main_logic     # Imports the core bot cycle logic

# --- Control ---
# How often to run the main logic cycle (in seconds) when looping
# Set to 0 if you want to run cycles purely manually by re-running the execution cell
RUN_INTERVAL_SECONDS = 5

# %% [markdown]
# ## 1. Initialize Session, Load State, Fetch Symbol Info
# Run this cell ONCE when starting or restarting the notebook kernel. Check output for errors.

# %%
# --- Initial Load State FIRST ---
# Load previous state before initializing (though init resets some state if no position)
bot_state = state_manager.load_state()

# --- Initialize Session and Get Symbol Info ---
print("--- Attempting Initialization ---")
session = None      # Will be populated by successful initialization
symbol_info = None  # Will be populated by successful initialization
initialized_ok = False # Flag to track success

# Call the main initialization function from bybit_client
# It returns True/False for success, and the symbol_info dict on success
init_success, fetched_symbol_info = bybit_client.initialize_session_and_config()

if init_success:
    print("--- Initialization SUCCESSFUL ---")
    session = bybit_client.get_session() # Get the successfully created session
    symbol_info = fetched_symbol_info  # Store the fetched symbol info
    initialized_ok = True
    # Optional: Reload state again AFTER successful init if needed?
    # bot_state = state_manager.load_state() # Usually load at start is fine
else:
    print("--- Initialization FAILED ---")
    # Session will remain None, bot cannot run

# --- Final Check ---
if not session or not symbol_info:
     print("CRITICAL: Session or Symbol Info is missing after initialization attempt. Bot cannot run.")
     initialized_ok = False
else:
     # Ensure bot_state exists even if file load failed but init worked
     if not bot_state:
         bot_state = state_manager.load_state() # Try loading again or use initial
     print("Ready to run bot cycle.")
     # Display initial state loaded/set
     print(f"Initial Bot State: {bot_state}")


# %% [markdown]
# ## 2. Run Bot Cycle(s)
# Execute the cell below.
# - If `RUN_INTERVAL_SECONDS > 0` (set in first cell), it will loop indefinitely (Press Kernel -> Interrupt to stop).
# - If `RUN_INTERVAL_SECONDS = 0`, it will run only ONE cycle each time you execute it.

# %%
if initialized_ok:
    if RUN_INTERVAL_SECONDS > 0:
        print(f"Starting continuous loop (checking every {RUN_INTERVAL_SECONDS} seconds)... Press Kernel -> Interrupt to stop.")
        try:
            while True:
                # --- Execute One Cycle ---
                # Pass the current state and get the potentially modified state back
                bot_state = main_logic.run_bot_cycle(session, bot_state, symbol_info)

                print(f"Loop finished. Waiting {RUN_INTERVAL_SECONDS} seconds...")
                time.sleep(RUN_INTERVAL_SECONDS)
        except KeyboardInterrupt:
            print("\nLoop interrupted by user. Saving final state...")
            state_manager.save_state(bot_state) # Save state on interrupt
            print("State saved. Exiting loop.")
        except Exception as e:
            print(f"!!! UNHANDLED ERROR in main loop: {e}")
            print("Saving state before potentially stopping...")
            state_manager.save_state(bot_state)
            # Depending on error, might want to break or continue after delay
            print(f"Loop stopped due to error.")
    else:
        # --- Execute One Cycle Manually ---
        print("Running a single bot cycle...")
        # Pass the current state and get the potentially modified state back
        bot_state = main_logic.run_bot_cycle(session, bot_state, symbol_info)
        print("Single cycle complete. Run this cell again for next cycle.")
        # State is saved within run_bot_cycle when changes occur
else:
    print("Bot cannot run because session or symbol info failed to initialize in the previous cell.")

# %% [markdown]
# ## 3. (Optional) Manual State Save
# You can run this cell to manually save the current state if needed. State is also saved automatically during the cycle when changes happen (like SO placement or resets).

# %%
# if 'bot_state' in locals() and bot_state and initialized_ok:
#     print("Manually saving state...")
#     state_manager.save_state(bot_state)
#     print("State saved.")
# elif not initialized_ok:
#     print("Cannot save state, initialization failed.")
# else:
#     print("Bot state not available to save (run initialization first).")

INFO: Using estimated total fee buffer for TP: 0.1100%
State loaded from bot_state_3m.json
--- Attempting Initialization ---
Attempting to initialize Bybit HTTP session...
Bybit session initialized and API keys seem valid.
Attempting to set leverage for BTCUSDT to 10x...
Request → POST https://api-testnet.bybit.com/v5/position/set-leverage: {"category": "linear", "symbol": "BTCUSDT", "buyLeverage": "10", "sellLeverage": "10"}.
Fetching Instrument Info...
Symbol Info OK: PPrec=2, QPrec=3, MinQ=0.001, MaxQ=1190.0, Step=0.001
--- Initialization SUCCESSFUL ---
Ready to run bot cycle.
Initial Bot State: {'filled_safety_orders_count': 0, 'calculated_base_qty_this_cycle': 0.001, 'active_safety_order_ids': [], 'is_in_trade': True, 'current_trade_entry_timestamp': 1745277875540, 'current_trade_side': 'Sell', 'current_trade_avg_price': 79800.0, 'pending_entry_order_id': None, 'pending_order_placed_timestamp': 0, 'tsl_activated_this_trade': False}
Starting continuous loop (checking every 5 second